# Основы Python. Часть 2.2

## Итераторы, генераторы, исключения


---

### 0. "Iterable object"
Питоновский объект "итерируемый" – означает, что  в цикле можно пройтись по каждому элементу этого объекта.

In [1]:
my_list = ['a', 'b', 'c', 'd', 'e']  # простой список
my_list

['a', 'b', 'c', 'd', 'e']

In [2]:
type(my_list)

list

Обход списка в цикле тремя способами:

In [3]:
# C/C++ style
for i in range(len(my_list)):
    print(my_list[i])

a
b
c
d
e


In [4]:
# python style
for element in my_list:
    print(element)

a
b
c
d
e


In [5]:
# combined style
for i, element in enumerate(my_list):
    print(i, element)

0 a
1 b
2 c
3 d
4 e


Аналогично работает для кортежей (tuple), словарей, множеств, строк и т.д.

Но не работает например для чисел:

In [6]:
a = 5
for number in a:
    print(number)

TypeError: 'int' object is not iterable

**Вопрос**: почему именно такое поведение и как это реализовано в Python?

**Ответ**: итерируемые классы поддерживают так называемый *протокол итераций* – *iterator protocol* (об этом ниже).

---
---

### 1. Выражения генераторов списков (list comprehension expression)

Способы создания списков:
*  вручную

In [7]:
my_list = [1, 2, 3, 4, 5]
my_list

[1, 2, 3, 4, 5]

*  поэлементным добавлением в цикле или функции

In [8]:
my_list = []
for i in range(1, 6):
    my_list.append(i)
my_list

[1, 2, 3, 4, 5]

*  с помощью выражения генераторов списков

In [9]:
my_list = [i for i in range(1, 6)]
my_list

[1, 2, 3, 4, 5]

Можно считать, что две предыдущие записи эквивалентны.

 ---

В чем преимущества создания списков с помощью "List Comprehension":
1. Читаемость кода
2. Скорость выполнения
3. Возможность использовать более сложные выражения (с условиями и итерированием по нескольким спискам)

In [10]:
def create_list_with_append(list_length=1000000):
    my_list = []
    for i in range(1, list_length+1):
        my_list.append(i)
    return my_list

In [11]:
def create_list_with_comprehension(list_length=1000000):
    return [i for i in range(1, list_length+1)]

In [12]:
%timeit create_list_with_append();

58.4 ms ± 536 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%timeit create_list_with_comprehension();

31.3 ms ± 109 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


 Примеры:

In [14]:
a = zip(['a', 'b', 'c'], [1, 2, 3])
a, list(a)

(<zip at 0x7f423410a308>, [('a', 1), ('b', 2), ('c', 3)])

In [15]:
[k*v for (k, v) in zip(['a', 'b', 'c'], [1, 2, 3])]

['a', 'bb', 'ccc']

In [16]:
[x ** 2 if x % 2 == 0 else x ** 3 for x in range(10)]

[0, 1, 4, 27, 16, 125, 36, 343, 64, 729]

In [17]:
first = []
for x in range(1, 5):
    for y in range(5, 1, -1):
        if x != y:
            first.append((x, y))
            
# эквивалентно:
            
[(x, y) for x in range(1, 5) for y in range(5, 1, -1) if x != y]

[(1, 5),
 (1, 4),
 (1, 3),
 (1, 2),
 (2, 5),
 (2, 4),
 (2, 3),
 (3, 5),
 (3, 4),
 (3, 2),
 (4, 5),
 (4, 3),
 (4, 2)]

Но помните! — __Simple is better than complex__ (Простое лучше, чем сложное)*

*PEP 20 -- The Zen of Python  
_import this_ в интерпретаторе  
https://www.python.org/dev/peps/pep-0020/  
https://tyapk.ru/blog/post/the-zen-of-python

 ---

Приятный бонус – аналогичный синтаксис существует для создания __словарей__ и __множеств__:

In [18]:
my_dict = {key: value for key, value in zip([1, 2, 3], ['a', 'b', 'c'])}
type(my_dict), my_dict

(dict, {1: 'a', 2: 'b', 3: 'c'})

In [19]:
my_set = {key for key in [1, 1, 2, 2, 3, 3]}
type(my_set), my_set

(set, {1, 2, 3})

---
---

### 2. Выражения-генераторы (generator expressions)

__Выражения-генераторы__ – напоминают генераторы списков, но они не конструируют список с
результатами, а возвращают объект, который будет воспроизводить результаты по требованию.  
Поскольку такая конструкция не создает сразу весь список с результатами, она позволяют
__экономить память и производить дополнительные вычисления между операциями__ получения результатов.  
Такая возможность возврата результатов по требованию обеспечена за счёт реализации _протокола итераций_.

In [20]:
my_gen = (k*v for (k, v) in zip(['a', 'b', 'c', 'd', 'e'], [1, 2, 3, 4, 5]))
my_gen

<generator object <genexpr> at 0x7f42340f2c50>

In [21]:
# Получаем сразу все значения с помощью преобразования в список:
list(my_gen)

['a', 'bb', 'ccc', 'dddd', 'eeeee']

In [22]:
# Второй раз не получится, т.к. данный генератор уже "выдал" все свои значения!
list(my_gen)

[]

In [23]:
# Если хотим пройтись по значениям ещё раз, то придётся создавать генератор заного:
my_gen = (k*v for (k, v) in zip(['a', 'b', 'c', 'd', 'e'], [1, 2, 3, 4, 5]))
tuple(my_gen)

('a', 'bb', 'ccc', 'dddd', 'eeeee')

In [24]:
# Также значения можно получать в цикле:
my_gen = (i for i in range(3))
for i in my_gen:
    print(i)

0
1
2


In [25]:
# Или передавая генератор стандартной функции next():
my_gen = (i for i in range(3))
next(my_gen)

0

In [26]:
next(my_gen)

1

In [27]:
next(my_gen)

2

In [28]:
# Но тогда, при исчерпании значений, будет вызвано исключение StopIteration:
next(my_gen)

StopIteration: 

 ---

 ### ВАЖНО! Значения генерируются не в момент создания генератора, а в момент его вызова!

In [29]:
# Пример:

my_list = [1, 2, 3, 4, 5]
my_gen = (i*i for i in my_list)

# далее может быть любой код...
# в том числе и изменяющий исходный список:
my_list.append(6)

# теперь получаем итоговые значения:
list(my_gen)

[1, 4, 9, 16, 25, 36]

Выражение-генератор использовало обновленный список.

В отличие от генераторов-списков, которые рассчитывают итоговые значения СРАЗУ:

In [30]:
# Пример:

my_list = [1, 2, 3, 4, 5]
my_list_gen = [i*i for i in my_list]

# далее идёт какой-то код...
# в том числе и изменяющий исходный список:
my_list.append(6)

# получаем итоговые значения:
list(my_list_gen)

[1, 4, 9, 16, 25]

 ---

Остальные свойства генераторов-выражений аналогичны генераторам списков  
(можно использовать в заглавии цикла, с условными выражениями и с итерациями по нескольким значениям):

In [31]:
my_gen = ((x, y) for x in range(1, 5) for y in range(5, 1, -1) if x != y)
for x, y in my_gen:
    print(x, y)

1 5
1 4
1 3
1 2
2 5
2 4
2 3
3 5
3 4
3 2
4 5
4 3
4 2


---
---

### 3. Функции-генераторы (generator functions)

Что если нужна более сложная логика для генераторов?  
Для этого в Python существуют функции-генераторы – они выглядят как обычные инструкции для создания функций _def_, но для возврата результатов по одному значению за раз используют команду __yield__ (вместо __return__), приостанавливающую выполнение функции в данном месте программы до следующего вызова значения генератора.

In [32]:
# Пример обычной функции:
def get_random_numbers(N_numbers, a=1, b=100):
    from random import randint  # генерирует случайное целое число в диапазоне от 'a' до 'b'
    
    random_numbers_list = []
    
    for i in range(int(N_numbers)):
        random_numbers_list.append(randint(int(a), int(b)))
        
    return random_numbers_list  # здесь возвращается весь список

In [33]:
my_list = get_random_numbers(5)
my_list

[49, 1, 40, 54, 38]

In [34]:
# Аналогичная функция-генератор:
def get_random_numbers_generator(N_numbers, a=1, b=100):
    from random import randint  # генерирует случайное целое число в диапазоне от 'a' до 'b'
    
    for i in range(int(N_numbers)):
        random_number = randint(int(a), int(b))
        print('Получено число: {}'.format(random_number))
        
        yield random_number  # здесь происходит возврат одного значения и заморозка состояния до следующего раза
        print('Продолжаю генерацию...')
        
    print('Генерация случайных чисел окончена', end='\n\n')

In [35]:
random_numbers = get_random_numbers_generator(5)
random_numbers

<generator object get_random_numbers_generator at 0x7f423408e990>

In [36]:
print(list(random_numbers))

Получено число: 18
Продолжаю генерацию...
Получено число: 51
Продолжаю генерацию...
Получено число: 22
Продолжаю генерацию...
Получено число: 18
Продолжаю генерацию...
Получено число: 7
Продолжаю генерацию...
Генерация случайных чисел окончена

[18, 51, 22, 18, 7]


In [37]:
# По прежнему генератор сработает только один раз (если он не бесконечный – но тогда (в данном коде) он бы завис):
print(list(random_numbers))

[]


In [38]:
# Само собой можно получать значения в цикле:
random_numbers = get_random_numbers_generator(5)

for number in random_numbers:
    print(number)

Получено число: 37
37
Продолжаю генерацию...
Получено число: 63
63
Продолжаю генерацию...
Получено число: 45
45
Продолжаю генерацию...
Получено число: 36
36
Продолжаю генерацию...
Получено число: 8
8
Продолжаю генерацию...
Генерация случайных чисел окончена

